In [15]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [16]:
df = pd.read_csv('data/cannabis.csv')

In [17]:
df = df.replace('None', np.nan)
df = df.dropna().reset_index()

In [18]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
dtm = tfidf.fit_transform(df['Effects'])
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)

In [19]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [20]:
# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
NN_model = pickle.load(model_pkl)
print ("Loaded model :: ", NN_model)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [21]:
import pickle
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [22]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [23]:
import json
def recommend(user_input):
    temp_df = NN_model.kneighbors(tfidf_model.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.iloc[temp_df[0][i]]['Strain']
        info_effects = df.iloc[temp_df[0][i]]['Effects']
        info_flavor = df.iloc[temp_df[0][i]]['Flavor']
        info_description = df.iloc[temp_df[0][i]]['Description']
        info_rating = df.iloc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [24]:
recommend('relieving stress and muscle tension')

"Club-69"
"Sleepy,Relaxed"
"Diesel,Flowery,Earthy"
"Club 69 is a special strain crafted by Josh D of Karma Genetics for Arizona\u2019s own, Sunday Goods. This Kushy cross of Karmarado OG and Biker Kush is a knockout for consumers looking for a heavy, restful indica-dominant strain. The terpene profile is a mixture of gas and pine, leaving the consumer with clean floral notes on the exhale. Enjoy this strain with care as the effects have been known to be powerfully sedative with continued consumption."
5.0
"Dawgfather-Og"
"Sleepy,Relaxed"
"Spicy/Herbal,Pepper,Pungent"
"Dawgfather OG, or simply \u201cThe Dawgfather,\u201d is an indica-dominant hybrid created by AlphaKronik Genes capable of massive plant production and a healthier-than-average yield. With potency floating above 20%, this strain contends well with chronic pain, insomnia, Crohn's, and anxiety. Expect traditional notes of gas/chemicals, Kush, and some funkiness. Dawgfather OG offers a strong buzz with nuance worth exploring.